In [ ]:
import numpy as np

from tqdm import tqdm
from utils import zscore_masked_items, get_rmse_score
from fancyimpute import SoftImpute, BiScaler

## SoftImpute

In [ ]:
class SoftImputeBiscaler():

    def __init__(self, A, max_rank=3, max_iters=10, n_power_iterations=100):
        self.A = A
        self.max_rank = max_rank
        self.max_iters = max_iters
        self.n_power_iterations = n_power_iterations
        self.solver = None
        self.scaler = None
        self.rec_A = None

    def train(self, test_matrix=None):
        error_progress = {
            "train_rmse": [],
            "test_rmse": [],
        }
        self.solver = SoftImpute(max_rank=self.max_rank, max_iters=self.max_iters,
                                 n_power_iterations=self.n_power_iterations
                                 )
        self.scaler = BiScaler(min_value=1, max_value=5)
        
        data_normalized = self.scaler.fit_transform(np.copy(self.A))
        prediction_norm = self.solver.fit_transform(data_normalized)
        self.rec_A = self.scaler.inverse_transform(prediction_norm)
        
        train_rmse = get_rmse_score(self.rec_A, self.A)
        error_progress["train_rmse"].append(train_rmse)
        if test_matrix is not None:
            test_rmse = get_rmse_score(self.rec_A, test_matrix)
            error_progress["test_rmse"].append(test_rmse)
        # print(error_progress)
        return error_progress

    def reconstruct_matrix(self):
        """
        Compute the full matrix using the solver
        """
        return self.rec_A

In [ ]:
# A, test_matrix = import_data_to_matrix_split()
# model = SoftImputeBiscaler(max_rank=3, max_iters=10, n_power_iterations=100)
# model.train(test_matrix=test_matrix)

In [ ]:
A = import_data_to_matrix()
model = SoftImputeBiscaler(max_rank=3, max_iters=10, n_power_iterations=100)
model.train()
rec_A = model.reconstruct_matrix()

In [ ]:
rec_A[rec_A>5] = 5
rec_A[rec_A<1] = 1

In [ ]:
extract_submission(rec_A, file="softimpute")